<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Evolutionary_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deap

In [ ]:
from deap import base, creator, tools, algorithms
import random

# Define the fitness function (maximize sum of individual values)
def eval_function(individual):
    return sum(individual),

# Create the necessary DEAP classes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximizing fitness
creator.create("Individual", list, fitness=creator.FitnessMax)

# Set up the toolbox for the genetic algorithm
toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)  # Generate random float (0.0 - 1.0)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 10)  # Individual = 10 random floats
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # Population = list of individuals
toolbox.register("mate", tools.cxTwoPoint)  # Two-point crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Bit-flip mutation (probability = 0.05)
toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection
toolbox.register("evaluate", eval_function)  # Evaluation function

# Initialize the population and Hall of Fame
pop = toolbox.population(n=100)  # Population size = 100
hof = tools.HallOfFame(1)  # Keep the best individual

# Run the genetic algorithm
algorithms.eaSimple(
    population=pop,
    toolbox=toolbox,
    cxpb=0.7,  # Crossover probability
    mutpb=0.2,  # Mutation probability
    ngen=50,  # Number of generations
    stats=None,
    halloffame=hof,
    verbose=True
)

# Output the best individual after the algorithm is complete
print("Best individual:", hof[0])
print("Best fitness:", hof[0].fitness.values[0])